In [1]:
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot

### Set up camera and robot, load pre-trained SSD model for COCO

In [2]:
model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
robot = Robot()

In [3]:
# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)
mtx = camera_cal['mtx']
dist = camera_cal['dist']

# Open Image Widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)

BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)

In [47]:
start_bbox_TV = [(102, 90), (161, 134)]

pos1_bbox_TV = [(102, 90), (164, 135)]
pos1_bbox_stool = [(168, 123), (200, 182)]

pos2_bbox_TV = [(99, 79), (170, 128)]
pos2_bbox_stool = [(171, 115), (211, 183)]

pos2left_bbox_TV = [(136, 64), (218, 132)]
pos2left_bbox_stool = [(215, 96), (283, 187)]

pos2right_bbox_TV = [(51, 69), (134, 131)]
pos2right_bbox_stool = [(131, 122), (164, 184)]

pos3_bbox_TV = [(90, 72), (172, 127)]
pos3_bbox_stool = [(172, 107), (219, 189)]

# Bbox for TV and stool at various camera poses
beg_bbox_TV = [(72, 92), (136, 137)]  # Pose (0,0,0)
mid_bbox_TV = [(33, 69), (114, 124)]  # Pose (0.5,0,0)
mid_bbox_stool = [(175, 125), (216, 183)]   # Pose (0.5,0,0)
end_bbox_stool = [(164, 101), (217, 185)]   # Pose (1.0,0,0)

In [5]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image (optional)
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def draw_bbox(img, width, height, bbox, color, line_width):
    bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                  (int(width * bbox[2]), int(height * bbox[3]))]
    cv2.rectangle(img, bbox_pixel[0], bbox_pixel[1], color, line_width)
    return bbox_pixel

def display_detected(img, detections, width, height, debug=False):
    """ put blue bounding boxes on detected objects on image """
    
    for det in detections[0]:
        label = COCO_labels[det['label']]
        bbox = det['bbox']
        bbox_pixel = draw_bbox(img, width, height, bbox, BLUE, 1)
        if debug:
            print(label,det['label'], bbox_pixel)
    return

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections, debug= False):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if debug:
            print(center)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points from a bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL(upper left), UR(upper right), LR(lower right), LL(lower left) feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]

def generate_ref_obj_features(ref_obj, img, detections, width, height, debug=False):
    """ Find reference object in detected objects and generate 4 feature points """
    matching_detections = [d for d in detections[0] if d['label'] == ref_obj]

    # get detection closest to center of field of view and draw it (if model detects multiple
    # reference objects)
    det = closest_detection(matching_detections)

    if det is not None:
        bbox = det['bbox']
        # bound reference object in green
        draw_bbox(img, width, height, bbox, GREEN, 1)
        # convert to pixel units
        bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]
        return create_feature_pts(bbox_pixel) # return 4 feature points of bounding box
    else:
        return None
    
def fp_error(current, goal):
    """ Return error of a feature point """
    current_x, current_y = current
    goal_x, goal_y = goal
    # Use Corke's convention of (p*-p) --> gain is a +ve number
    return (goal_x-current_x, goal_y-current_y)  

def features_error(currentFeatures, goalFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_goal, UR_goal, LR_goal, LL_goal = goalFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_goal)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_goal)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_goal)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_goal)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))

def image_jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0, u/Z, u*v/f_u, -(f_u+u*u/f_u), v],
                  [0, -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v, -u]])
    return L

def robot2world(robot_orientation):
    """ calculate the robot jacobian """
    theta = robot_orientation
    
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])

def control2robot(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    return np.array([[r/2, r/2],
                  [r/l, -r/l]])

In [74]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 2.0
robot_theta = math.pi/8 

# robot parameters
wheel_radius = 0.0325
axle_length = 0.12

# Control loop parameters
gain = 0.1
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 50
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True
"""
# Use stool as reference object
ref_obj = 62
goal_features = create_feature_pts(end_bbox_stool)
print(ref_obj,COCO_labels[ref_obj], goal_features)

"""
# Use TV as reference object 
ref_obj = 72
goal_features = create_feature_pts(mid_bbox_TV)
print(ref_obj,COCO_labels[ref_obj], goal_features)


start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=False) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        break
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error)
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(robot_theta)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    
    if i%interval == 0 and debug:
        print(wheel_velocities)
        print(robot_velocities)
    
    w_r = wheel_velocities[0,0]
    w_l =wheel_velocities[1,0]
    
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
  
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

72 laptop [(33, 69), (114, 69), (114, 124), (33, 124)]
Reference object not detected!!!
FPS: 314.4


In [60]:
wheel_velocities[1,0]

0.27418986728719835